In [1]:
from feature.make_dataset import make_data
from feature.after_dataset import accumulate
from feature.moving_average import moving_average
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')


c:\Python\Python38\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [3]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)
train_rol = moving_average(train_input_list)
test_rol = moving_average(test_input_list)

In [4]:
train = train.reset_index()
test = test.reset_index()
train_rol = train_rol.reset_index()
test_rol = test_rol.reset_index()

In [5]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적총광량')

In [6]:
train = pd.concat([train, train_rol], axis=1)
test = pd.concat([test, test_rol], axis=1)

In [7]:
drop_feature = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량', 'index']
train = train.drop(drop_feature, axis=1)
test = test.drop(drop_feature, axis=1)


In [8]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from catboost import CatBoostRegressor
X = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
y = train['predicted_weight_g']





In [9]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'depth': trial.suggest_int('depth', 10, 16),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 3e-5, log=True),
        'verbose': 500,
        "random_state": 42,
    }

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
    cat = CatBoostRegressor(**params, task_type='GPU')
    cat.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds=50,
            verbose=500)
    cat_pred = cat.predict(X_valid)
    rmse = mean_squared_error(y_valid, cat_pred)**0.5

    return rmse


sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name='cat_parameter_opt',
    direction='minimize',
    sampler=sampler,
)

study.optimize(objective, n_trials=50)
print("Best Score:", study.best_value)
print("Best trial", study.best_trial.params)

[I 2022-12-07 11:59:13,873] A new study created in memory with name: cat_parameter_opt


: 

: 

In [ ]:
X = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
y = train['predicted_weight_g']
from sklearn.model_selection import StratifiedKFold, KFold

skf = KFold(n_splits=5, shuffle=True, random_state=404)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

CAT_model = {}

for f in range(5):
    print(f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    CAT = CatBoostRegressor(verbose=500, early_stopping_rounds=50, task_type="GPU", random_seed=113)
    CAT.fit(x_train, y_train)

    y_pred = CAT.predict(x_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    print(f"{f + 1} Fold RMSE = {rmse}")
    CAT_model[f] = CAT
    print(f'================================================================================\n\n')


submit = test[['DAT', 'predicted_weight_g']]
x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
for fold in range(5):
    submit['predicted_weight_g'] += CAT_model[fold].predict(x_test)/5
all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
for idx, test_path in enumerate(all_target_list):
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
    submit_df.to_csv(test_path, index=False)

====================================1============================================
Learning rate set to 0.037997
0:	learn: 41.5685024	total: 19ms	remaining: 18.9s
500:	learn: 4.1934766	total: 8.87s	remaining: 8.84s
999:	learn: 3.3854897	total: 17.6s	remaining: 0us
1 Fold RMSE = 8.70409110085975


====================================2============================================
Learning rate set to 0.037997
0:	learn: 39.9937046	total: 18ms	remaining: 18s
500:	learn: 4.7553083	total: 9.07s	remaining: 9.03s
999:	learn: 3.8379899	total: 18.1s	remaining: 0us
2 Fold RMSE = 7.80194374542085


====================================3============================================
Learning rate set to 0.037997
0:	learn: 41.1764326	total: 19.1ms	remaining: 19s
500:	learn: 4.4940597	total: 9.14s	remaining: 9.11s
999:	learn: 3.4376989	total: 18.3s	remaining: 0us
3 Fold RMSE = 7.272555397149389


====================================4============================================
Learning rate set to 0.03799

In [101]:
df_imp = pd.DataFrame({'imp':CAT.feature_importances_}, index = CAT.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
"('ec관측치', 7)",0.000103
water,0.000145
"('co2관측치', 17)",0.000549
"('co2관측치', 19)",0.001197
"('일간누적분무량', 8)",0.001519
...,...
"('내부온도관측치', 15)",2.412617
월간누적총광량,5.505976
월간누적적색광량,12.467472
월간누적백색광량,18.082506


In [102]:
df_imp.to_csv('zz.csv')